In [12]:
pip install langchain==0.0.179 faiss-cpu==1.7.4

In [1]:
import langchain
langchain.__version__

'0.0.179'

# Loading

In [20]:
from langchain.document_loaders import WebBaseLoader

url = 'https://raw.githubusercontent.com/Aliraqimustafa/state_of_the_union/main/state_of_the_union.txt'
web_loader = WebBaseLoader(url)
docs_loaded = web_loader.load()

In [21]:
len(docs_loaded)

1

In [22]:
print(docs_loaded[0].page_content[:50])

Madam Speaker, Madam Vice President, our First Lad


# Splitting

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 700, chunk_overlap = 100)

docs_splitted = text_splitter.split_documents(docs_loaded)

In [24]:
len(docs_splitted)

64

In [26]:
print(docs_splitted[0].page_content)

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.


# Storage [ VectorStores ]

In [37]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

embeddings = OpenAIEmbeddings()
storage = FAISS.from_documents(documents = docs_splitted ,embedding = embeddings )

In [50]:
iraq_docs = storage.similarity_search_with_score(query = 'What was said about Iraq?', k=2)

In [54]:
print(iraq_docs[1])

(Document(page_content='As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  \n\nAnd we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  \n\nPutin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. \n\nAnd a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.', metadata={'source': 'https://raw.githubusercontent.com/Aliraqimustafa/state_of_the_union/main/state_of_the_union.txt'}), 0.44413096)


# LLM

In [59]:
from langchain.llms.openai import OpenAI
from langchain.chains.question_answering import load_qa_chain

llm = OpenAI(temperature=.0)

chain = load_qa_chain(llm=llm)

In [81]:
template = chain.llm_chain.prompt.template
print(template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


# Combine LLM & Storage

In [92]:
def Aswer_question(user_question ,k=5):
    docs = storage.similarity_search(user_question,k=k)
    return chain.run(input_documents=docs, question=user_question)

## Test

In [93]:
answer = Aswer_question('What was said about Iraq?')

In [95]:
print(answer)

 Our troops in Iraq and Afghanistan faced many dangers, one of which was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. When they came home, many of the world’s fittest and best trained warriors were never the same.
